In [1]:
import os
import pandas as pd
import numpy as np

pd.options.display.max_columns = 100

head_dir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games/')
data_dir = os.path.join(head_dir, 'Data/3_lrn/Raw')

files = [f for f in os.listdir(data_dir) if f[-3:] == 'csv']
file_dict = dict(zip([f[:-4] for f in files], np.arange(len(files))))

In [2]:
sessions = ['a', '2', 'b', '4', 'c']
sesshes = [f[8:-4].split('_') for f in files]
sesharray = np.array(sesshes)
subjects = np.unique(sesharray[:, 1])
subject_map = dict(zip(subjects, np.arange(len(subjects))))

In [3]:
top = 184                                                  # board bounds and dimensions in pixels
bottom = 587
left = 227
right = 1138
width = right - left
height = bottom - top

def mouse_x_to_tile(x):
    """Converts mouse x coordinates to board-space"""
    return 9*(x - left) / width

def mouse_y_to_tile(y):
    """Converts mouse y coordinates to board-space"""
    return 4*(y - top) / height

def fix_game_boards(row):
    """Removes move from appropriate color board string representation for a single record"""

    bp, wp = row[['bp', 'wp']]                              # select board string reps
    if row['color']==0:                                     # if Black is player
        p = list(bp)                                        # convert board string to list (mutability)
        p[int(row['zet'])] = '0'                            # set list at zet loc to be '0'
    else:                                                   # if White is player, do the same thing for White's board
        p = list(wp)
        p[int(row['zet'])] = '0'

    return ''.join(p)                                       # rejoin into new string

def load_mouse_file(mf):
    """Loads and preprocesses mouse tracking data"""

    mfnames = [
        'idx', 'id', 'color', 'gi', 'mi',
        'status', 'bp', 'wp', 'zet',
        'rt', 'ts', 'mt', 'mx'
    ]                                                      # names for columns

    D = pd.read_csv(mf, names=mfnames)                     # load csv into pandas dataframe
    D = D.loc[pd.notnull(D['ts'])]
    D['mx'] = D.apply(expand_mouse_mx, axis=1)             # append start and end mouse spatial coords
    D['mt'] = D.apply(expand_mouse_mt, axis=1)             # append start and end mouse timestamps
    D = D[['mt', 'mx']]                                    # lose the fluff

    valid = pd.notnull(D['mt'])                            # select records with valid mouse time series
    m = (D.loc[valid, 'mt'] + ',').sum().split(',')[:-1]   # combine all mouse timestamp records
    x = [tuple(xy.split(',')) for xy in (D.loc[valid, 'mx'] + ';').sum().split(';')][:-1]
                                                           # combine all mouse coordinate records
    M = pd.DataFrame(index=m, data=x, columns=['x', 'y'])  # new dataframe with timestamp index and coordinates
    M['subject'] = mf[-6:-4]                               # set subject field to initials
    M['human'] = M['subject'].map(file_dict)     # set human field to subject ordinal index
    M.index = M.index.astype(int)                          # cast timestamp index to integers
    return M

def load_game_file(gf):
    """Loads and preprocesses data from game observations"""

    gfnames = [
        'idx', 'id', 'color', 'gi', 'mi',
        'status', 'bp', 'wp', 'zet',
        'rt', 'ts', 'mt', 'mx'
    ]                                                      # names for columns

    D = pd.read_csv(gf, names=gfnames)                     # load csv into pandas dataframe
    readyfilter = D['status'] == 'ready'                   # filter on convenience records
    rtfilter = D['rt'] == 0                                # filter on AI moves
    aifilter = ~(readyfilter & rtfilter)                   # filter out convenience AI records
    D = D.loc[aifilter]                                    # apply filter
    D.loc[readyfilter, 'rt'] = 0                           # set human convenience records rt field to 0
    D['subject'] = gf[-6:-4]                               # set subject field to initials in game file name
    D['human'] = D['subject'].map(file_dict)     # set human field to be subject index (alphabetical)
    D.loc[pd.notnull(D['ts']), 'move_start_ts'] = (D.loc[pd.notnull(D['ts']), 'ts'].astype(int) - D.loc[pd.notnull(D['ts']), 'rt'].astype(int)).shift(-1)     # set move_start_timestamp field to turn beginning
    tsfilter = rtfilter | readyfilter                      # filter on ai OR convenience records
    D.loc[tsfilter, 'ts'] = D.loc[tsfilter, 'move_start_ts']
                                                           # replace invalid timestamps with inferred correct timestamp
    D.loc[pd.notnull(D['ts']), 'mx'] = D.loc[pd.notnull(D['ts'])].apply(expand_mouse_mx, axis=1)             # append move start and end mouse spatial coords
    D.loc[pd.notnull(D['ts']), 'mt'] = D.loc[pd.notnull(D['ts'])].apply(expand_mouse_mt, axis=1)             # append move start and end timestamps to mouse timestamps
    D['is human'] = 1                                      # initialize human player indicator variable
    playfilter = D['status'].isin(['playing', 'win', 'draw'])
                                                           # filter on non-convenience records
    D.loc[playfilter & rtfilter, 'is human'] = 0           # set human player indicator to 0 on AI records
    endfilter = D['status'].isin(['win', 'draw'])          # filter on game end records
    idx = D.loc[endfilter].index                                   # get indices for game end filter application
    if D.loc[idx[-1], 'rt'] != 0:                          # if human player ended last game
        D.loc[idx[-1], 'gi'] = D.loc[idx[-1], 'gi'] - 1    # subtract 1 from game index (why? probably a data error)
    bpfilter = D['color']==0                               # filter on player colors
    wpfilter = D['color']==1
    D.loc[bpfilter, 'bp'] = D.loc[bpfilter].apply(fix_game_boards, axis=1)
    D.loc[wpfilter, 'wp'] = D.loc[wpfilter].apply(fix_game_boards, axis=1)
                                                           # apply filters and remove last move from board
    return D.set_index('ts')                               # set index to timestamps

def expand_mouse_mt(row):
    """Appends start time, end time to mouse timestamp records for a single record"""

    endtime = int(row['ts'])                               # get turn end timestamp
    starttime = endtime - int(row['rt'])                   # get turn start from turn end and turn duration
    if type(row['mt'])==str:                               # check if valid data
        return str(starttime) + ',' + row['mt'] + ',' + str(endtime)
                                                           # add start, end times to respective ends of record
def expand_mouse_mx(row):
    """Appends start time location, end time location to mouse spatial coordinates for a single record"""

    endtime = int(row['ts'])                               # get turn end timestamp
    starttime = endtime - int(row['rt'])                   # get turn start from turn end and turn duration
    if type(row['mx'])==str:                               # check if valid data
        locs = row['mx'].split(';')                        # split record into (x, y) pair strings
        endloc = locs[-1]                                  # select first and last coordinate pairs
        startloc = locs[0]
        return startloc + ';' + row['mx'] + ';' + endloc    # add start, end coords to respective ends of record

def mouse_hist(m, g):
    """Modifies mousetracking data to produce histograms over tile indices"""

    g['turn'] = 100*g['gi'] + g['mi']                      # add unique turn ids
    turnfilter = g['status'].isin(['playing', 'draw', 'win'])
                                                           # filter on non-convenience records
    m['turn'] = np.nan                                     # initialize helper fields
    m['turnstart'] = np.nan
    m['turnend'] = np.nan
    m['ts'] = m.index
    m['xtile'] = np.nan
    m['ytile'] = np.nan
    m['tile'] = np.nan
    m['dur'] = np.nan
    m['is human'] = np.nan
    m = m.drop_duplicates(subset='ts')                     # get rid of duplicate timestamps
    validfilter = g.index.isin(m.index)
    gp = g.loc[validfilter & turnfilter]
    m.loc[gp.index, 'turn'] = gp['turn']                   # add helper data to mouse df
    m.loc[gp.index, 'turnstart'] = gp.index - gp['rt']
    m.loc[gp.index, 'turnend'] = gp.index
    m.loc[gp.index, 'is human'] = gp['is human']

    m = m.sort_index()                                     # sort mouse data by timestamp
    fillthese = ['turn', 'turnstart', 'turnend', 'is human']
                                                           # helper columns to fill
    m[fillthese] = m[fillthese].fillna(method='bfill')     # backfill missing data

    m['dur'] = m.index
    m['dur'] = m['dur'].diff(periods=1)                    # compute duration of each event
    eventbounds = (m.index > m['turnstart']) & (m.index <= m['turnend'])
                                                           # filter on mouse data within player turn
    m = m.loc[eventbounds]                                 # apply filter

    m['xtile'] = m['x'].astype(float).map(mouse_x_to_tile) # map mouse coords to board coords
    m['ytile'] = m['y'].astype(float).map(mouse_y_to_tile)
    m['tile'] = m['xtile'].astype(int) + 9*m['ytile'].astype(int) # compute mouse tile

    humanfilter = m['is human'] == 1                       # filter on human moves (mouse df)
    mpvt = m.loc[humanfilter].pivot_table(index='turn', columns='tile', values='dur', aggfunc=np.sum)
                                                           # pivot human trials duration per tile idx
    mpvt['rt'] = mpvt.sum(axis=1)                          # recalculate rt for verification

    offboard = [
        i for i in mpvt.columns
        if (i not in list(range(36)) and type(i)==int)
    ]                                                      # column names for offboard locs

    mpvt[999] = mpvt[offboard].sum(axis=1)                 # combine all offboard durations
    humanfilter = g['is human'] == 1                       # filter on human moves (game df)
    turnfilter = g['status'].isin(['playing', 'draw', 'win'])
    gt = g.loc[turnfilter & humanfilter & validfilter].set_index('turn') # get non-convenience human records
    mpvt.loc[gt.index, 'true rt'] = gt['rt']               # set 'true rt' for verification
    mpvt = mpvt.fillna(value=0)                            # nan values mean 0 duration

    for c in ['bp', 'wp', 'zet']:
        mpvt.loc[gt.index, c] = gt[c]                      # set other game info fields on hist records

    for c in range(36):
        if c not in mpvt.columns:                          # set all nonvisited trials to 0 dur
            mpvt[c] = 0

    return m, mpvt

In [13]:
output_dir = os.path.join(head_dir, 'Analysis/3_lrn/mouse_histograms')
game_data = {}
mouse_data = {}
mouse_hists = {}
for i, subject in enumerate(subjects):
    print('\n', subject, '\n')
    for j, session in enumerate(sessions):
        print(session)
        if subject not in game_data.keys():
            game_data[subject] = {}
            mouse_data[subject] = {}
            mouse_hists[subject] = {}
        gdfile = 'learning{session}_{subject}.csv'.format(session=session, subject=subject)
        G = load_game_file(os.path.join(data_dir, gdfile))
        game_data[subject][session] = G
        M = load_mouse_file(os.path.join(data_dir, gdfile))
        
        top = M.y.astype(int).min()
        bottom = M.y.astype(int).max()
        left = M.x.astype(int).min()
        right = M.x.astype(int).max()
        
        mouse_data[subject][session], H = mouse_hist(M, G)
        mouse_hists[subject][session] = H
        Mfilename = '{subject}_{session}.csv'.format(subject=subject, session=session)
        H.to_csv(os.path.join(output_dir, Mfilename))


 AB 

a


/Users/gianni/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2
b
4
c

 AB2 

a
2
b
4
c

 CE 

a
2
b
4
c

 EN 

a
2
b
4
c

 EW 

a
2
b
4
c

 GB 

a
2
b
4
c

 GL 

a
2
b
4
c

 HS 

a
2
b
4
c

 IK 

a
2
b
4
c

 IS 

a
2
b
4
c

 JC 

a
2
b
4
c

 JP 

a
2
b
4
c

 KL 

a
2
b
4
c

 LH 

a
2
b
4
c

 LM 

a
2
b
4
c

 LW 

a
2
b
4
c

 MS 

a
2
b
4
c

 OR 

a
2
b
4
c

 QC 

a
2
b
4
c

 SG 

a
2
b
4
c

 SL 

a
2
b
4
c

 SW 

a
2
b
4
c

 TQ 

a
2
b
4
c

 VC 

a
2
b
4
c

 XG 

a
2
b
4
c

 XM 

a
2
b
4
c

 XW 

a
2
b
4
c

 XZ 

a
2
b
4
c

 YV 

a
2
b
4
c

 YZ 

a
2
b
4
c


In [ ]:
        
game_data = [load_game_file(os.path.join(data_dir, file)) for file in files]
mouse_data = [load_mouse_file(os.path.join(data_dir, file)) for file in files]

pvts = []
for subject in range(len(mouse_data)):
    top = mouse_data[subject].y.astype(int).min()                       # board bounds and dimensions in pixels
    bottom = mouse_data[subject].y.astype(int).max()
    left = mouse_data[subject].x.astype(int).min()
    right = mouse_data[subject].x.astype(int).max()
    width = right - left
    height = bottom - top
    
    
    mouse_data[subject], mpvt = mouse_hist(mouse_data[subject], game_data[subject])
    pvts.append(mpvt)